In [ ]:
import itertools
import math
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option('display.max_column', None)

# Context

In [ ]:
data = pd.read_csv("../data/train.csv")
data.head()
# Notice: embarked 	Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton
# Notice: sibsp 	# of siblings / spouses aboard the Titanic
# Notice: parch 	# of parents / children aboard the Titanic

# Data quality assessment

In [ ]:
data.info()

## Перечисления

In [ ]:
assert data['PassengerId'].is_unique
assert 0 == data[~data['Survived'].isin((0, 1))].size
data['Survived'] = data['Survived'].astype('bool')
assert 0 == data[~data['Sex'].isin(('male', 'female'))].size
assert 0 == data[~data['Embarked'].isin(['C', 'Q', 'S', np.nan])].size
assert 0 == data[~data['Pclass'].isin([1, 2, 3])].size

## Проверим возможные полные дубликаты.

In [ ]:
assert 0 == data[data.duplicated(subset=list(filter(lambda c: c != 'PassengerId', data.columns)))].size

## Родственные связи
У супружеской пары без детей или у пары сиблингов в 'SibSp' должно быть по единице, следовательно, при согласованных данных количество семей из N человек можно вычислить как M / (SibSp + 1), где M - количество записей SibSp == N - 1.

In [ ]:
_d = data.groupby('SibSp').count()['PassengerId'].reset_index()
_d[_d['PassengerId'] % (_d['SibSp'] + 1) != 0]

Вывод: для кого-то количество супругов/сиблингов указанно с ошибкой.
Аналогично проверим родиетелй/детей.

In [ ]:
_d = data.groupby('Parch').count()['PassengerId'].reset_index()
_d[_d['PassengerId'] % (_d['Parch'] + 1) != 0]

Тоже не сходится.

## Разберём сложные строки: букквенные коды могут быть полезны.

In [ ]:
data[['TicketPref', 'TicketNum']] = data['Ticket'].str.extract(r'(?:(.+)\s)?(\d+)')
# Нулевых билетов нет, поэтому можно заменить NaN на 0
# TODO: data[data['TicketNum'].isnull()] = 0
data['TicketNum'] = data['TicketNum'].astype('float64')
data[['CabinPref', 'CabinNum']] = data['Cabin'].str.extract(r'([A-Za-z])(\d+)')
# Нулевых кают нет, поэтому можно заменить NaN на 0
# TODO: data[data['CabinNum'].isnull()] = 0
data['CabinNum'] = data['CabinNum'].astype('float64')

Поскольку здесь нет полностью бесполезных строк, ничего удалять не будем. При необходимости данные будем фильтровывать, исключая кортежи с пустыми значениями.
Для работы с возрастом может быть интересна классификация на взрослый/ребёнок.

In [ ]:
def get_type(age: int, sex: str) -> str:
    if age >= 60:
        a = 'Old'
    elif age >= 18:
        a = 'Adult'
    else:
        a = 'Child'
    s = sex.capitalize()
    return f'{a}{s}'

data['type'] = data.apply(lambda x: get_type(x.Age, x.Sex), axis=1)

# Data exploration

## Отдельные параметры

In [ ]:
data.describe()

Явно аномальных значений (например, возраст в 1000 лет) нет.

In [ ]:
corr_b_fields = ['Survived']
corr_n_fields = ['Pclass', 'Age', 'Parch', 'SibSp', 'Fare', 'TicketNum', 'CabinNum']
corr_o_fields = ['TicketPref', 'CabinPref', 'Embarked', 'type']

for f in corr_o_fields:
    plt.subplots()
    sns.countplot(data=data[data[f].notnull()], x=f)
for f in corr_n_fields:
    plt.subplots()
    sns.histplot(data=data[data[f].notnull()], x=f)

Здесь речь о тех пассажирах для которых известен параметр.
Префикс каюты - вероятно палуба. Числовая часть билетов явно имеет повторы, причины неизвестны. Стоимость проезда имеет три выраженных пика, вероятно, связана с пассажирским классом. Интересны пики в гистограмме возраста, стоит рассмотреть связи. Наибольшее количество пассажиров в диапазоне 20-30 лет. Больше всего пассажиров зашло на борт в Саутгемптоне. Преобладающий тип пассажира - взрослый мужчина, класс - третий.

In [ ]:
#data.groupby('Age').count().sort_values(by='PassengerId', ascending=False)

In [ ]:
data.corr(method="pearson")

Наблюдаем слабую связь выживаемости с возрастом (0.103895), стоимостью билета (0.187534) и среднюю связь с пассажирским классом (-0.289723).
Стоимость билета сильно связана (-0.417354) с пассажирским классом (очевидно, 1-й дороже).

Рассмотрим корреляцию разных переменных. Участки кода, отображающие все возможные сочетания закомментированы, чтобы не перегружать отчёт менее малоинформативными графиками. По умолчанию отображаются только показавшиеся информативными.

In [ ]:
#for a, b in itertools.combinations(corr_o_fields + corr_b_fields, 2):
#    sns.catplot(y=a, hue=b, kind="count", data=data)

Рассмотрим влияние на выживание пола и возраста.

In [ ]:
sns.catplot(y='type', hue='Survived', kind="count", data=data)

Во всех возрастных категориях чаще выживают женщины.
На выживаемость пассажиров наверняка влиет планировка палуб и рапсределение на них пассажиров.

In [ ]:
#sns.catplot(y='CabinPref', hue='Pclass', kind="count", data=data)
#sns.catplot(y='type', hue='Survived', kind="count", data=data)
#sns.catplot(y='Pclass', hue='Survived', kind="count", data=data)
#sns.scatterplot(x='Fare', y='')
sns.heatmap((data.groupby(['CabinPref', 'Pclass']).count()).reset_index().pivot(index='CabinPref', columns='Pclass', values='PassengerId'), annot=True)
plt.subplots()
sns.heatmap((data[data['Survived']].groupby(['CabinPref', 'Pclass']).count() / data.groupby(['CabinPref', 'Pclass']).count()).reset_index().pivot(index='CabinPref', columns='Pclass', values='PassengerId'), annot=True)

Как видно, наибольший процент выживших среди пассажиров второго класса палуб G и F.

## Бонус. Спасают ли от утопления "счастливые билеты"?

In [ ]:
lucky_data = data[data['TicketNum'].notnull()][['Survived', 'TicketNum']]
def is_lucky(n):
    n = str(int(n))
    return sum(map(int, n[:math.floor(len(n)/2)])) == sum(map(int, n[math.ceil(len(n)/2):]))

lucky_data['is_lucky'] = lucky_data['TicketNum'].apply(is_lucky)
lucky_alive = lucky_data[lucky_data['is_lucky'] & lucky_data['Survived']].count().values[0] / lucky_data[lucky_data['is_lucky']].count().values[0]
unlucky_alive = lucky_data[~lucky_data['is_lucky'] & lucky_data['Survived']].count().values[0] / lucky_data[~lucky_data['is_lucky']].count().values[0]
print(f"Выживших со счастливым билетом {int(lucky_alive*100)}%, без счастливого {int(unlucky_alive*100)}%.")

Вывод: "счастливые билетики" от утопления *не* спасают.

# Summary